In [1]:
import tensorflow as tf
import os
import numpy as np
from glob import glob as glb
from tqdm import *
import h5py

min_queue_examples = 1000

/Users/ichang-geon/.pyenv/versions/2.7.14/envs/flow/lib/python2.7/site-packages/h5py/__init__.py:36: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from ._conv import register_converters as _register_converters
/Users/ichang-geon/.pyenv/versions/2.7.14/envs/flow/lib/python2.7/site-packages/h5py/__init__.py:45: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from . import h5a, h5d, h5ds, h5f, h5fd, h5g, h5r, h5s, h5t, h5p, h5z
/Users/ichang-geon/.pyenv/versions/2.7.14/envs/flow/lib/python2.7/site-packages/h5py/_hl/group.py:22: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from .. import h5g, h5i, h5o, h5r, h5t, h5l, h5p


In [2]:
def load_flow(filename, shape):
  stream_flow = h5py.File(filename, 'r')
  flow_state_vel = np.array(stream_flow['Velocity_0'][:])
  flow_state_vel = flow_state_vel.reshape([shape[0], shape[1]+128, 3])[0:shape[0],0:shape[1],0:2]
  stream_flow.close()
  return flow_state_vel

def load_boundary(filename, shape):
  stream_boundary = h5py.File(filename, 'r')
  boundary_cond = np.array(stream_boundary['Gamma'][:])
  boundary_cond = boundary_cond.reshape([shape[0], shape[1]+128, 1])[0:shape[0],0:shape[1],:]
  stream_boundary.close()
  return boundary_cond

def read_data(filename_queue, shape):
  reader = tf.TFRecordReader()
  key, serialized_example = reader.read(filename_queue)
  features = tf.parse_single_example(
    serialized_example,
    features={
      'boundary':tf.FixedLenFeature([],tf.string),
      'sflow':tf.FixedLenFeature([],tf.string),
    }) 
  boundary = tf.decode_raw(features['boundary'], tf.uint8)
  sflow = tf.decode_raw(features['sflow'], tf.float32)

  boundary = tf.reshape(boundary, [shape[0], shape[1], 1])
  sflow = tf.reshape(sflow, [shape[0], shape[1], 2])
  boundary = tf.to_float(boundary)
  sflow = tf.to_float(sflow)
  return boundary, sflow

def _generate_image_label_batch(boundary, sflow, batch_size, shuffle=True):
  num_preprocess_threads = 1
  boundarys, sflows = tf.train.shuffle_batch(
    [boundary, sflow],
    batch_size=batch_size,
    num_threads=num_preprocess_threads,
    capacity=min_queue_examples + 3 * batch_size,
    min_after_dequeue=min_queue_examples)
  return boundarys, sflows

def flow_inputs(batch_size):
  shape = (128,256)

  tfrecord_filename = glb('./data/*.tfrecords') 
  
  filename_queue = tf.train.string_input_producer(tfrecord_filename) 

  boundary, sflow = read_data(filename_queue, shape)

  boundarys, sflows = _generate_image_label_batch(boundary, sflow, batch_size)
 
  return boundarys, sflows



In [3]:
n_batch = 8
learning_rate = 0.0001
keep_prob = 0.7

global_step = tf.Variable(0, trainable=False, name='global_step')

X = tf.placeholder(tf.float32 , [None,128,256,1], name="X")

boundary, sflow = flow_inputs(n_batch)
X = boundary
# sflow = tf.placeholder(tf.float32 ,[n_batch, 128, 256, 2])
# keep_prob = tf.placeholder(tf.float32)

#Conv1
W1 = tf.Variable(tf.random_normal([16, 16, 1, 128], stddev=0.01))
L1 = tf.nn.conv2d(X, W1, strides=[1, 8, 16, 1], padding='SAME')
L1 = tf.nn.relu(L1)
L1 = tf.nn.dropout(L1, keep_prob)

In [4]:
#Cov2
W2 = tf.Variable(tf.random_normal([4,4,128,512], stddev=0.01))
L2 = tf.nn.conv2d(L1, W2, strides=[1, 4, 4, 1], padding='SAME')
L2 = tf.nn.relu(L2)
L2 = tf.nn.dropout(L2, keep_prob)

In [5]:
L2_flat = tf.reshape(L2,[-1,4*4*512])

In [6]:
W3 = tf.get_variable("W", shape=[512 * 4 * 4, 1024],
                     initializer=tf.contrib.layers.xavier_initializer())
b3 = tf.Variable(tf.random_normal([1024]))
L3 = tf.nn.relu(tf.matmul(L2_flat, W3) + b3)
L3 = tf.nn.dropout(L3, keep_prob=keep_prob)

In [7]:
# L3 = tf.divide(L3,y)
S1, S2 = tf.split(L3, [512, 512], 1)

In [8]:
S1= tf.reshape(S1,[n_batch,1,1,512])
S2= tf.reshape(S2,[n_batch,1,1,512])

In [9]:
# deconv1
W4_1 = tf.Variable(tf.random_normal([8,8,512,512], stddev=0.01))
L4_1 = tf.nn.conv2d_transpose(S1,W4_1,output_shape=[n_batch,8,8,512],strides=[1,8, 8, 1], padding='SAME')
L4_1 = tf.nn.relu(L4_1)
L4_1 = tf.nn.dropout(L4_1, keep_prob)

W4_2 = tf.Variable(tf.random_normal([8,8,512,512], stddev=0.01))
L4_2 = tf.nn.conv2d_transpose(S2,W4_2,output_shape=[n_batch,8,8,512],strides=[1,8, 8, 1], padding='SAME')
L4_2 = tf.nn.relu(L4_2)
L4_2 = tf.nn.dropout(L4_2, keep_prob)

In [10]:
## deconv2
W5_1 = tf.Variable(tf.random_normal([4,8,256,512], stddev=0.01))
L5_1 = tf.nn.conv2d_transpose(L4_1,W5_1,output_shape=[n_batch,32,64,256],strides=[1, 4, 8, 1], padding='SAME')
L5_1 = tf.nn.relu(L5_1)
L5_1 = tf.nn.dropout(L5_1, keep_prob)

W5_2 = tf.Variable(tf.random_normal([4,8,256,512], stddev=0.01))
L5_2 = tf.nn.conv2d_transpose(L4_2,W5_2,output_shape=[n_batch,32,64,256],strides=[1,4, 8, 1], padding='SAME')
L5_2 = tf.nn.relu(L5_2)
L5_2 = tf.nn.dropout(L5_2, keep_prob)

In [11]:
# deconv3
W6_1 = tf.Variable(tf.random_normal([2,2,32,256], stddev=0.01))
L6_1 = tf.nn.conv2d_transpose(L5_1,W6_1,output_shape=[n_batch,64,128,32],strides=[1,2, 2, 1], padding='SAME')
L6_1 = tf.nn.relu(L6_1)
L6_1 = tf.nn.dropout(L6_1, keep_prob)

W6_2 = tf.Variable(tf.random_normal([2,2,32,256], stddev=0.01))
L6_2 = tf.nn.conv2d_transpose(L5_2,W6_2,output_shape=[n_batch,64,128,32],strides=[1,2, 2, 1], padding='SAME')
L6_2 = tf.nn.relu(L6_2)
L6_2 = tf.nn.dropout(L6_2, keep_prob)

In [12]:
# deconv4
W7_1 = tf.Variable(tf.random_normal([2,2,1,32], stddev=0.01))
L7_1 = tf.nn.conv2d_transpose(L6_1,W7_1,output_shape=[n_batch,128,256,1],strides=[1,2,2, 1], padding='SAME')
L7_1 = tf.nn.dropout(L7_1, keep_prob)

W7_2 = tf.Variable(tf.random_normal([2,2,1,32], stddev=0.01))
L7_2 = tf.nn.conv2d_transpose(L6_2,W7_2,output_shape=[n_batch,128,256,1],strides=[1,2, 2, 1], padding='SAME')
L7_2 = tf.nn.dropout(L7_2, keep_prob)

In [13]:
sflow_p = tf.stack([L7_1 , L7_2] , axis=3)
sflow_p = tf.reshape(sflow_p , [n_batch,128,256,2])
# loss = tf.reduce_mean(tf.square(sflow_p - sflow))
loss = tf.nn.l2_loss(sflow_p - sflow)

In [14]:
#loss 
total_loss = tf.train.AdamOptimizer(learning_rate).minimize(loss)

In [15]:
sess = tf.Session()
saver = tf.train.Saver(tf.global_variables())
ckpt = tf.train.get_checkpoint_state('./model_save')
if ckpt and tf.train.checkpoint_exists(ckpt.model_checkpoint_path):
    saver.restore(sess, ckpt.model_checkpoint_path)
else:
    sess.run(tf.global_variables_initializer())

INFO:tensorflow:Restoring parameters from ./model_save/model.ckpt


In [16]:
tf.train.start_queue_runners(sess=sess)

[<Thread(QueueRunnerThread-input_producer-input_producer/input_producer_EnqueueMany, started daemon 123145479098368)>,
 <Thread(QueueRunnerThread-shuffle_batch/random_shuffle_queue-shuffle_batch/random_shuffle_queue_enqueue, started daemon 123145483304960)>]

In [17]:
steps = 200

for epoch in range(steps):    
  total_cost = 0
  _, cost_val = sess.run([total_loss, loss],feed_dict={})
  total_cost += cost_val

  print('Epoch:', '%04d' % (epoch + 1),
        'Avg. cost =', '{:f}'.format(total_cost))
print("최적화 완료") 
saver.save(sess, "./model_save/model.ckpt")

('Epoch:', '0001', 'Avg. cost =', '558.317383')
('Epoch:', '0002', 'Avg. cost =', '488.307739')
('Epoch:', '0003', 'Avg. cost =', '526.650879')
('Epoch:', '0004', 'Avg. cost =', '608.244568')
('Epoch:', '0005', 'Avg. cost =', '588.740967')
('Epoch:', '0006', 'Avg. cost =', '501.632385')
('Epoch:', '0007', 'Avg. cost =', '521.304565')
('Epoch:', '0008', 'Avg. cost =', '525.813293')
('Epoch:', '0009', 'Avg. cost =', '518.190491')
('Epoch:', '0010', 'Avg. cost =', '530.894409')
('Epoch:', '0011', 'Avg. cost =', '517.703003')
('Epoch:', '0012', 'Avg. cost =', '547.473877')
('Epoch:', '0013', 'Avg. cost =', '510.237640')
('Epoch:', '0014', 'Avg. cost =', '516.588440')
('Epoch:', '0015', 'Avg. cost =', '498.728394')
('Epoch:', '0016', 'Avg. cost =', '634.264038')
('Epoch:', '0017', 'Avg. cost =', '512.181946')
('Epoch:', '0018', 'Avg. cost =', '575.085205')
('Epoch:', '0019', 'Avg. cost =', '569.132080')
('Epoch:', '0020', 'Avg. cost =', '575.311218')
('Epoch:', '0021', 'Avg. cost =', '481.1

('Epoch:', '0172', 'Avg. cost =', '474.422577')
('Epoch:', '0173', 'Avg. cost =', '416.472961')
('Epoch:', '0174', 'Avg. cost =', '441.841797')
('Epoch:', '0175', 'Avg. cost =', '426.889862')
('Epoch:', '0176', 'Avg. cost =', '443.376312')
('Epoch:', '0177', 'Avg. cost =', '488.439972')
('Epoch:', '0178', 'Avg. cost =', '468.937531')
('Epoch:', '0179', 'Avg. cost =', '455.163452')
('Epoch:', '0180', 'Avg. cost =', '437.426422')
('Epoch:', '0181', 'Avg. cost =', '419.555756')
('Epoch:', '0182', 'Avg. cost =', '438.830322')
('Epoch:', '0183', 'Avg. cost =', '448.127869')
('Epoch:', '0184', 'Avg. cost =', '463.155121')
('Epoch:', '0185', 'Avg. cost =', '427.075806')
('Epoch:', '0186', 'Avg. cost =', '391.239990')
('Epoch:', '0187', 'Avg. cost =', '449.031464')
('Epoch:', '0188', 'Avg. cost =', '420.329132')
('Epoch:', '0189', 'Avg. cost =', '410.043793')
('Epoch:', '0190', 'Avg. cost =', '438.009644')
('Epoch:', '0191', 'Avg. cost =', '446.936859')
('Epoch:', '0192', 'Avg. cost =', '465.7

'./model_save/model.ckpt'